## Paso 6: Técnica del Árboles de Decisión

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
import time
import itertools

# Se cargan los datos en un DataFrame
df_datos = pd.read_csv("datos/credit_risk_dataset.csv")

# Imputar valores ausentes utilizando la media para las columnas numéricas y la moda para las categóricas
imputer = SimpleImputer(strategy='mean')  # Para columnas numéricas
df_datos[['person_age', 'person_income', 'cb_person_cred_hist_length']] = imputer.fit_transform(df_datos[['person_age', 'person_income', 'cb_person_cred_hist_length']])

# Se imputa la moda para 'loan_status' y 'loan_grade'
imputer = SimpleImputer(strategy='most_frequent')  # Para columnas categóricas
df_datos[['loan_status', 'loan_grade']] = imputer.fit_transform(df_datos[['loan_status', 'loan_grade']])

# Crear una instancia de OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Ajustar y transformar la columna 'loan_grade' usando OrdinalEncoder
encoded_loan_grade = ordinal_encoder.fit_transform(df_datos[['loan_grade']])

# Crear un DataFrame con la codificación ordinal de 'loan_grade'
encoded_loan_grade = pd.DataFrame(encoded_loan_grade, columns=['loan_grade'])

# Eliminar la columna original 'loan_grade' y concatenar la nueva codificación ordinal
df_datos = pd.concat([df_datos.drop(columns=['loan_grade']), encoded_loan_grade], axis=1)

# Lista de todas las columnas
todas_columnas = ['person_age', 'person_income', 'cb_person_cred_hist_length', 'loan_grade']

def train_evaluate_model(X_train, X_test, y_train, y_test, columns, max_depth, criterion):
    columns = list(columns)  # Convertir columns a lista
    
    # Se crea y entrena el modelo de clasificación del árbol de decisión
    clasificador_arbol_decision = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth)
    modelo_clasificador_arbol_decision = clasificador_arbol_decision.fit(X_train[columns], y_train)

    # Se predicen las salidas de datos de prueba
    y_test_predicho = modelo_clasificador_arbol_decision.predict(X_test[columns])

    # Se calculan las métricas para evaluar el modelo de clasificación
    mc = confusion_matrix(y_test, y_test_predicho)
    e = accuracy_score(y_test, y_test_predicho)
    p = precision_score(y_test, y_test_predicho, zero_division=0)  # Se agrega zero_division=0 para manejar la advertencia
    s = recall_score(y_test, y_test_predicho)
    f1 = f1_score(y_test, y_test_predicho)

    return mc, e, p, s, f1

# Se crea el conjunto de datos de entrenamiento y de prueba con todas las columnas
X = df_datos[todas_columnas]
y = df_datos['loan_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Se almacenan los resultados en una lista
resultados = []

# Se define la lista de configuraciones de árbol de decisión
configuraciones_arbol_decision = [(2, 'entropy'), (10, 'entropy')]

# Se itera sobre cada combinación posible de columnas
for max_depth, criterion in configuraciones_arbol_decision:
    for r in range(1, len(todas_columnas) + 1):
        for combination in itertools.combinations(todas_columnas, r):
            start_time = time.time()
            # Se entrena y evalúa el modelo con la configuración actual
            mc, e, p, s, f1 = train_evaluate_model(X_train, X_test, y_train, y_test, combination, max_depth, criterion)
            end_time = time.time()

            # Se calcula el tiempo de ejecución
            execution_time = end_time - start_time

            # Se almacenan los resultados en la lista de resultados
            resultados.append([combination, max_depth, criterion, mc, e, p, s, f1, execution_time])

# Se crea el DataFrame final
df_resultados_arbol_decision = pd.DataFrame(resultados, columns=['Columnas', 'Max Depth', 'Criterion', 'Matriz de Confusión', 'Exactitud', 'Precisión', 'Sensibilidad', 'F1 Score', 'Tiempo de Ejecución'])

# Se muestra el DataFrame final
print(df_resultados_arbol_decision)


                                             Columnas  Max Depth Criterion  \
0                                       (person_age,)          2   entropy   
1                                    (person_income,)          2   entropy   
2                       (cb_person_cred_hist_length,)          2   entropy   
3                                       (loan_grade,)          2   entropy   
4                         (person_age, person_income)          2   entropy   
5            (person_age, cb_person_cred_hist_length)          2   entropy   
6                            (person_age, loan_grade)          2   entropy   
7         (person_income, cb_person_cred_hist_length)          2   entropy   
8                         (person_income, loan_grade)          2   entropy   
9            (cb_person_cred_hist_length, loan_grade)          2   entropy   
10  (person_age, person_income, cb_person_cred_his...          2   entropy   
11            (person_age, person_income, loan_grade)          2